In [1]:
import numpy as np
import random
import matplotlib
matplotlib.use('Agg')  # noqa
import matplotlib.pyplot as plt

In [2]:
# Input Params

## Human Params ##
human_death_rate = 1/36500  # 1 human death every 100 yrs (for now, inaccurate)
inf_human_death_rate = 0.05/14  # An infected person dies in 7 days (for now, inaccurate)

## Mosquito Params ## 
mean_bites = 0.75  # average proportion of mosquitoes that successfully bite at each time step
stdev_bites = 0.01  # stdev of mean_bites
mosquito_death_rate = 0.05  # mosquito life span is ~10 days

## Infection Params ##
protection = 0.05  # proportion of reduced risk to infection after one infection
p_human_inf = 0.19  # avg prob that someone contractions malaria from an infected mosquito bite
p_mosquito_inf = 0.417  # prob that mosquito contracts malaria from an infected human
human_recovery_time = 14
human_immunity_time = 28

In [3]:
# Initializing Peoples Array based off Params
def init_peoples(Nh, Ih, avg_age=0):
    peoples = np.array([['person', 'status', 'inf_age', 'prev_infs', 'days since recovered', 'age']])

    # Generate array
    for h in range(Nh):
        # Creating a random age w/ average of ave_age days
        # rand_age = np.random.normal(avg_age, 1825)
        # rand_age = np.round(rand_age).astype(int)
        new_person = [h+1, 0, 0, 0, 0, avg_age]  # Assuming no one in the pop has been infected before?
        peoples = np.vstack([peoples, new_person])

    # If we choose to start the pop with initial infection
    for i in range(Ih):
        people_list = list(range(1, Nh+1))
        rand_person = random.choice(people_list)
        # An already infected person is chosen
        while int(peoples[rand_person][1]) == 1:  # NOTE: I want the first row to be string, but that means all entries are strings
            # Pick a new person
            rand_person = random.choice(people_list)

        # Person isn't already infected
        if int(peoples[rand_person][1]) == 0:
            peoples[rand_person][1] = 1
            # random generate an age of infection
            rand_age = np.random.normal(7, 4)
            rand_age = np.round(rand_age).astype(int)
            peoples[rand_person][2] = rand_age
    return peoples

In [4]:
def mosquito_bite_sim(mean_bites, stdev_bites, Nh, Nm):
    mosquito_bite_frac = np.random.normal(mean_bites, stdev_bites)  # rand generate proportion of mosquitoes that bite
    mosquito_bite_frac = max(0, min(1, mosquito_bite_frac))  #if mosquito_bite_frac is less than 0 --> 0, and if mosquito_bite_frac > 1 --> 1
    mosquito_bites = np.round(mosquito_bite_frac*Nm).astype(int)  # find total N of mosquitoes that bite
    bitten_list = [random.randint(1, Nh) for _ in range(mosquito_bites)]  # generate random list containing values 1 to Nh that represents the people that have been randomly bitten
    return bitten_list

In [5]:
# INITIALIZE POPULATION
Nh = 1500  # Total Human Population size
Ih = 0  # Num of Infected Humans to start

Nm = 750  # Total Mosquito Pop Size
Im = 50  # Num of Infected Mosquitoes to start

peoples = init_peoples(Nh, Ih)

## Time Params ##
t_max = 3650*5

Ih_vs_t = [Ih/Nh]
Im_vs_t = [Im]

malaria_death_stat_prev_infs = []
malaria_death_stat_age = []

s_end_time_stat_prev_infs = []
s_end_time_stat_age = []
i_end_time_stat_prev_infs = []
i_end_time_stat_age = []
r_end_time_stat_prev_infs = []
r_end_time_stat_age = []

for t in range(t_max):
    Ih_count = 0
    # print(f'******************************* DAY: {t} *******************************')
    healthy_bitten_list = mosquito_bite_sim(mean_bites, stdev_bites, Nh, Nm-Im)
    infected_bitten_list = mosquito_bite_sim(mean_bites, stdev_bites, Nh, Im)
    # print(f'People bitten by healthy mosquitoes: {healthy_bitten_list}')
    # print(f'People bitten by infected mosquitoes: {infected_bitten_list}')
    for h in peoples[1:]:  # iterate all rows, excluding the first row bc that is just headers
        # INFECTED HUMAN RECOVERY
        if int(h[2]) == human_recovery_time:
            h[1] = '2'
            h[2] = '0'
            new = int(h[3]) + 1
            h[3] = str(new)
        
        # RECOVERED HUMAN LOSING IMMUITY
        if int(h[4]) == human_immunity_time:
            h[1] = '0'
            h[4] = '0'
        
        # INFECTED HUMAN IS BITTEN BY HEALTHY MOSQUITO
        if int(h[0]) in healthy_bitten_list and int(h[1]) == 1:
            # print('\nA HEALTHY MOSQUITO BIT AN INFECTED HUMAN')
            rand_contraction = np.random.rand()  # gen rand num between 0 and 1
            if rand_contraction <= p_mosquito_inf:
                Im = Im + 1
                # print(f'MOSQUITO HAS MALRIA NOW, Infected Mosquito Count = {Im}')
    
        # UNINFECTED HUMAN IS BITTEN BY INFECTED MOSQUITO
        if int(h[0]) in infected_bitten_list and int(h[1]) == 0:
            # print('\n\nUNINFECTED HUMAN IS BITTEN BY INFECTED MOSQUITO'
            n_prev_inf = int(h[3])  # find num of prev infections
            protection_factor = 1-(1-protection)**n_prev_inf
            # print(f'Person {h[0]}s num of prev infections = {n_prev_inf}, Protection factor = {protection_factor}')
            rand_contraction = np.random.rand()  # gen rand num between 0 and 1
            if rand_contraction <= p_human_inf*(1-protection_factor):
                h[1] = '1'  # person becomes infected
                # print(f'PERSON {h[0]} HAS MALARIA NOW')

        # HUMAN AGING UP
        new_age = int(h[5]) + 1
        h[5] = str(new_age)
        
        # INCREMENT DAYS SINCE RECOVERED
        if int(h[1]) == 2:  # if person is Recovered
            h[4] = str(int(h[4])+1)
        
        # HUMAN DEATH FROM MALARIA
        if int(h[1]) == 1:
            rand_death = np.random.rand()
            # infected person dies and replaced with a fresh susceptible
            if rand_death <= inf_human_death_rate:
                malaria_death_stat_prev_infs.append(int(h[3]))
                malaria_death_stat_age.append(int(h[5])/365)
                h[1] = '0'
                h[2] = '0'
                h[3] = '0'
                h[5] = '0'
                
            else:  # infected person lives to see another day
                new_inf_age = int(h[2])+1
                h[2] = str(new_inf_age)

        # RANDOM HEALTHY HUMAN DEATH
        else:
            rand_death = np.random.rand()
            if rand_death <= human_death_rate:
                h[1] = '0'
                h[2] = '0'
                h[3] = '0'
                h[4] = '0'

        if int(h[1]) == 1:
            Ih_count = Ih_count+1

    # MOSQUITO DEATH
    Im = (1-mosquito_death_rate)*Im
    Ih_vs_t.append(Ih_count/Nh)

for h in peoples[1:]:
    if int(h[1]) == 0:
        s_end_time_stat_prev_infs.append(int(h[3]))
        s_end_time_stat_age.append(int(h[5])/365)
    if int(h[1]) == 1:
        i_end_time_stat_prev_infs.append(int(h[3]))
        i_end_time_stat_age.append(int(h[5])/365)
    if int(h[1]) == 2:
        r_end_time_stat_prev_infs.append(int(h[3]))
        r_end_time_stat_age.append(int(h[5])/365)

In [6]:
t_list = [0]
for t in range(t_max):
    t_list.append(t+1)

fig, ax = plt.subplots()
ax.plot(t_list,Ih_vs_t)
ax.set_xlabel('Time (days)')
ax.set_ylabel('Human Infected Proportion (i(t))')
ax.spines[['right', 'top']].set_visible(False)
plt.savefig('i_vs_t1.png',dpi=300)

In [14]:
fig, ax = plt.subplots()
ax.scatter(malaria_death_stat_age, malaria_death_stat_prev_infs,alpha=0.5,color='#e30119')
ax.set_xlabel('Age of Person (Years)')
ax.set_ylabel("Number of Person's Previous Infections")
ax.spines[['right', 'top']].set_visible(False)
plt.savefig('MalariaDeathStats.png',dpi=300)


In [13]:
fig, ax = plt.subplots()
ax.scatter(s_end_time_stat_age,s_end_time_stat_prev_infs,alpha=0.5,label='Susceptible', color='#19e585')
ax.scatter(i_end_time_stat_age,i_end_time_stat_prev_infs,alpha=0.5,label='Infected', color='#e30119')
ax.scatter(r_end_time_stat_age,r_end_time_stat_prev_infs,alpha=0.5,label='Recovered', color = '#371ce6')
ax.set_xlabel("Age of Person (Years)")
ax.set_ylabel("Number of Person's Previous Infections")
ax.legend()
ax.spines[['right', 'top']].set_visible(False)
plt.savefig('EndTimesStats.png',dpi=300)